<a href="https://colab.research.google.com/github/juwon0502/NFL-Top-100/blob/main/Predictive_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import pandas as pd
from matplotlib import pyplot as plt

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as ln
from sklearn.linear_model import LogisticRegression as logn
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_text

In [141]:
MARK = 20

## Set up 

In [152]:
rank = pd.read_csv('https://raw.githubusercontent.com/juwon0502/NFL-Top-100/main/def_data.csv' , index_col=['Year','Player'])
qb = pd.read_csv('https://raw.githubusercontent.com/juwon0502/NFL-Top-100/main/qb_data.csv' , index_col=['Year','Player'])
wr = pd.read_csv('https://raw.githubusercontent.com/juwon0502/NFL-Top-100/main/wr_data.csv' , index_col=['Year','Player'])
te = pd.read_csv('https://raw.githubusercontent.com/juwon0502/NFL-Top-100/main/te_data.csv' , index_col=['Year','Player'])
rb = pd.read_csv('https://raw.githubusercontent.com/juwon0502/NFL-Top-100/main/rb_data.csv' , index_col=['Year','Player'])
deff = pd.read_csv('https://raw.githubusercontent.com/juwon0502/NFL-Top-100/main/def_data.csv' , index_col=['Year','Player'])

In [153]:
pm = pd.concat([wr,te,rb])

In [154]:
pm['Top'] = 0
pm.loc[pm['Rank']<=MARK , 'Top'] = 1
qb['Top'] = 0
qb.loc[qb['Rank']<=MARK , 'Top'] = 1
deff['Top'] = 0
deff.loc[deff['Rank']<= MARK , 'Top'] = 1

### Deffense

In [123]:
deff.groupby('Top').count()

,Rank,Position,Team,Points,Int,Yds,TD,Lng,PD,FF,Fmb,FR,Yds.1,TD.1,Sk,Comb,Solo,Ast,TFL,QBHits,Sfty,AV
Top,,,,,,,,,,,,,,,,,,,,,,
0,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394
1,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48


In [145]:
y = deff.Top
def_pos = ['DE', 'DT', 'LB', 'CB', 'S']

for pos in def_pos:
  deff[pos] = 0
  deff.loc[deff['Position'].str.match(pos) , pos] = 1

deff = deff.sort_values('Rank')
X = deff.drop(columns = ['Rank', 'Position', 'Team', 'Points', 'Top'])
X

,,Int,Yds,TD,Lng,PD,FF,Fmb,FR,Yds.1,TD.1,Sk,Comb,Solo,Ast,TFL,QBHits,Sfty,AV,DE,DT,LB,CB,S
Year,Player,,,,,,,,,,,,,,,,,,,,,,,
2014,J. J. Watt,1,80,1,80,10,4,0,5,59,1,20.5,78,59,19,29,51,1,21,1,0,0,0,0
2018,Aaron Donald,0,0,0,0,1,4,0,2,0,0,20.5,59,41,18,25,41,0,16,0,1,0,0,0
2016,Von Miller,0,0,0,0,3,3,0,0,0,0,13.5,78,62,16,13,24,0,15,0,0,1,0,0
2015,J.J. Watt,0,0,0,0,8,3,0,1,0,0,17.5,76,57,19,29,50,0,19,1,0,0,0,0
2019,Aaron Donald,0,0,0,0,2,2,0,1,0,0,12.5,48,29,19,20,24,1,13,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,Cameron Jordan,0,0,0,0,5,1,0,2,0,0,10.0,45,32,13,15,20,0,8,1,0,0,0,0
2016,Joey Bosa,0,0,0,0,0,0,0,0,0,0,10.5,41,29,12,17,21,0,5,1,0,0,0,0
2019,Lavonte David,1,26,0,26,7,3,0,1,0,0,1.0,123,82,41,10,7,0,8,0,0,1,0,0


In [146]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=0)

In [148]:
knn = KNN(n_neighbors=15)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
test_score = accuracy_score(y_pred, y_test)
print("test accuracy", test_score)
knn_pred = knn.predict(X_train)
train_score = accuracy_score(knn_pred, y_train)
print("training accuracy" , train_score)

test accuracy 0.821917808219178
training accuracy 0.8581081081081081


In [107]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Decision Tree

In [139]:
dtc = DTC(random_state=0, max_depth=3)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
test_score = accuracy_score(y_pred, y_test)
print("test accuracy", test_score)
knn_pred = dtc.predict(X_train)
train_score = accuracy_score(knn_pred, y_train)
print("training accuracy" , train_score)

test accuracy 0.8972602739726028
training accuracy 0.9391891891891891


In [140]:
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Playmakers (Wide Receivers, Running Backs, Tight Ends)

In [160]:
pm_pos = ['RB', 'WR', 'TE']
for pos in pm_pos:
  pm[pos] = 0
  pm.loc[pm['Position'].str.match(pos), pos] = 1

y = pm.Top
X = pm.drop(columns = ['Rank', 'Position', 'Team', 'Points', 'Top'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=0)

#### K-Nearest Neighbors

In [161]:
knn = KNN(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
test_score = accuracy_score(y_pred, y_test)
print("test accuracy", test_score)
knn_pred = knn.predict(X_train)
train_score = accuracy_score(knn_pred, y_train)
print("training accuracy" , train_score)

test accuracy 0.794392523364486
training accuracy 0.8564814814814815


#### Gausian Naive Bayes

In [162]:
gnb = GNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
test_score = accuracy_score(y_pred, y_test)
print("test accuracy", test_score)
knn_pred = gnb.predict(X_train)
train_score = accuracy_score(knn_pred, y_train)
print("training accuracy" , train_score)

test accuracy 0.8037383177570093
training accuracy 0.7962962962962963


#### Decision Tree

In [163]:
dtc = DTC(random_state=0, max_depth=3)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
print("test accuracy", test_score)
knn_pred = dtc.predict(X_train)
train_score = accuracy_score(knn_pred, y_train)
print("training accuracy" , train_score)

test accuracy 0.8037383177570093
training accuracy 0.875


#### Random Forest

In [164]:
rf = RFC(max_depth=3)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("test accuracy", test_score)
knn_pred = rf.predict(X_train)
train_score = accuracy_score(knn_pred, y_train)
print("training accuracy" , train_score)

test accuracy 0.8037383177570093
training accuracy 0.8935185185185185
